In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

# 1.Data Gethering

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/plenoi/Clinic/master/ultima_all_clean.csv')
df = df.set_index('hn')
df.head()

In [ ]:
df.shape

# 2.Data Pre-processing

In [ ]:
df.isnull().sum(axis=0)

In [ ]:
df_clean_column = df.drop(['appearance','Wardsize','RHlvsi','depth','nodeyiel','vgmargin','pelvicme','adnmet'],axis = 1)

In [ ]:
df_clean = df_clean_column.dropna(axis = 0)     ## Remove row ที่มี missing value

In [ ]:
df_clean.shape

In [ ]:
y = df_clean['pmmet'].values                          ## define class
X = df_clean.drop(['pmmet'],axis = 1).values

In [ ]:
allclass = np.unique(y)                                               ## check num of each class
allclassnumber = [sum(y==eachclass) for eachclass in allclass]
allclassnumber

# Imbalance

In [ ]:
from imblearn.over_sampling import SMOTE                                   ## ถ้า จน. class ไม่เท่ากันให้ใช้ smote
sm = SMOTE(random_state=1)
X_res, y_res = sm.fit_resample(X, y)

allclass = np.unique(y_res)
allclassnumber = [sum(y_res==eachclass) for eachclass in allclass]
allclassnumber

In [ ]:
from sklearn.model_selection import train_test_split                                                    ## Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.15, random_state=1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=1)      ## Train-Val Split

In [ ]:
from sklearn.preprocessing import MinMaxScaler         ## Normalization
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm =  scaler.transform(X_train)
X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_test)

# 3.Choose Model

# Deep Learning - Multiple Class

In [ ]:
import tensorflow as tf

In [ ]:
num_label = len(np.unique(y_train))
y_train_multi = (np.arange(num_label) == y_train[:,None]).astype(np.float32)
y_train_multi

y_val_multi = (np.arange(num_label) == y_val[:,None]).astype(np.float32)
y_val_multi

In [ ]:
tf.compat.v1.reset_default_graph()                                      # Clear Model   
np.random.seed(1)                                                       # Fix Randomness
tf.compat.v1.random.set_random_seed(1)                                  # Fix Randomness
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16, activation='relu', input_shape=(14,)),
  tf.keras.layers.Dense(8, activation='relu'),
  tf.keras.layers.Dense(4, activation='relu'),
  tf.keras.layers.Dense(2, activation='relu'),
  tf.keras.layers.Dense(num_label, activation='softmax')    
])
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_acc = list()                                    ## Train Model
val_acc = list()
for i in range(0,1000):
  history = model.fit(X_train_norm, y_train_multi, epochs= 1, batch_size = 200, validation_data= (X_val_norm, y_val_multi))
  tmp_avg = np.mean(history.history['accuracy'])
  tmp_avg_val = np.mean(history.history['val_accuracy'])
  train_acc.append(tmp_avg)
  val_acc.append(tmp_avg_val)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.figure(num=None, figsize=(16, 8), dpi=90, facecolor='w', edgecolor='k')
plt.plot()
plt.plot(train_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
y_predict = model.predict(X_test_norm)
y_predict = np.argmax(y_predict, 1)
sum(y_predict == y_test)/len(y_test)*100